In [355]:
pip install pymorphy3

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [356]:
pip install pyenchant

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [357]:
import re
import pymorphy3
from pymorphy3 import MorphAnalyzer
import enchant
import pandas as pd
import math
from collections import defaultdict

In [358]:
file  = open(r"C:\Users\779\Downloads\dataset_1937770_3.txt", mode='r')
lines = file.readlines()
file.close()
lines = lines[1:]
lines = [re.sub(r'^\d+,', '', line).rstrip('\r\n') for line in lines]
lines[0]

'куплюайфон14про'

1ый подход, Динамическое программирование без вероятностей, пробегаемся по строке, находим строки, которые можно считать словом(фильтр подбирал руками исходя из того как работал алгоритм), находим разбиение минимизирующее "стоимость" разбиения.

In [359]:
signs = {',', '.','"', '\'', ':', ';', '(', ')', '[', ']', '=', '+', '-', '*', '&', '^', '%', '$', '#', '@', '!','?'}
mph = pymorphy3.MorphAnalyzer()
english_dict = enchant.Dict("en_US")
BRANDS = {
    'iphone', 'samsung', 'google', 'apple', 'android', 'microsoft', 'nike', 'adidas',
    'tesla', 'bmw', 'mercedes', 'ikea'
}

def is_real_word(word, morph = mph, signs = signs):

    if word in signs or word.isdigit():
        return True

    if word.isalpha() and all(ord(c) < 128 for c in word):
        return english_dict.check(word.lower()) or word.lower() in BRANDS
    
    parsed = morph.parse(word)
    if not parsed:
        return False
    
    best = parsed[0]
    if best.score >= 0.15:
        for method in best.methods_stack:
            analyzer, dict_word = method[0], method[1]
            
            if "DictionaryAnalyzer" in str(analyzer):
                if dict_word.lower() == word.lower():
                    return True
    return False
        
    
def dinamic_separate(line):
    n = len(line)
    dp = []
    prev = []
    for i in range(0,n+1):
        if i == 0:
            dp.append(0)
            prev.append(-1)
        else:
            dp.append(100000)
            prev.append(0)
            for j in range(i):
                word = line[j:i]
                if is_real_word(word):
                    if dp[j] + 1 < dp[i]:  #стоимость разбиения = 1 сравниваем что выгоднее, разбить строку на j-ом симвле, или все первые i символов считать словом
                        dp[i] = dp[j] + 1
                        prev[i] = j
    positions = []
    i = n
    while i > 0:
        j = prev[i]
        if j != -1 and i != j and j != 0:  #
            positions.append(j) 
        i = j
    return sorted(positions)

Сохраняем в csv

In [360]:
def process_dataset(lines):

    df = pd.DataFrame({'id': range(len(lines)), 'input_string': lines})
    
    df['predicted_positions'] = df['input_string'].apply(
        lambda x: dinamic_separate(x)
    )
    
    df = df.drop(columns=['input_string'])
    
    return df


result_df = process_dataset(lines)

output_file = "output.csv"
result_df.to_csv(output_file, index=False, encoding='utf-8')



In [361]:
result_df.head()

,id,predicted_positions
0,0,"[5, 10, 12]"
1,1,"[3, 6, 7]"
2,2,"[4, 11, 13, 20, 21]"
3,3,"[5, 10, 18]"
4,4,"[5, 10]"


Значение F1 на платформе: 81.19%

Проблема: при ручном подборе score(вероятность того что pymorph3 правильно присвоил tag слову), пришлось допустить "плохие" слова: ив(ива), ев(ева), ав(ава) и тд, чтобы не упускать хорошие слова с тем же score: айфон.

In [362]:
dinamic_separate("купитьайфон14промакс") 

[6, 11, 13, 16]

In [363]:
dinamic_separate("книгавхорошемсостоянии") #тут видно что алгоритм неправильно разбил - книг ав хорошем состоянии, но стоимость разбиения та же

[4, 6, 13]

Чтобы попытаться побороться с проблемой(разбиваем строку на "странные" слова), изменил функцию стоимости (теперь это не просто число пробелов, а -сумма логарифмов вероятностей). Для этого нужны вероятности, я использую словарь с частотами, отдельно вероятностную модель не обучаю.

In [365]:
pip install wordfreq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [366]:
from wordfreq import word_frequency
import math

In [367]:
word_frequency("в", "ru")

0.0427

In [384]:
def word_cost(word):
    
    word = word.lower()
    
    if word in BRANDS:
        return 0.01
    
    if word.isdigit():
        return -math.log(0.001) / (len(word) + 0.01)
    #для чисел, чтобы последовательность цифр не разбивалась, нужно чтобы длинное число было "выгоднее" чем несколько отдельных подряд идущих цифр
    
    if word.isalpha() and all(ord(c) < 128 for c in word) and english_dict.check(word.lower()):
        return word_frequency(word, "en")
    
    if word.isalpha() and all(ord(c) < 128 for c in word):
        return 250 / (len(word) + 0.01)
    #В случае если не могли найти английское слово лучше просто отделить эту последовательность символов, но алгоритм за такое штрафуем
            
    prob = word_frequency(word, 'ru')
    
    if prob > 0:
        return -math.log(prob)
    elif is_real_word(word):
        return 15
    return 100  


In [378]:
word_cost('книга')

9.09701168666918

In [ ]:
#тут алгоритм почти не меняется, раньше стоимость cost слова был всегда равен 1, тут он вычисляется на основе частот
def dinamic_separate_freq(line):
    n = len(line)
    dp = []
    prev = []
    for i in range(0, n + 1):
        if i == 0:
            dp.append(0)
            prev.append(-1)
        else:
            dp.append(100000)  
            prev.append(0)
            for j in range(i):
                word = line[j:i]
                cost = word_cost(word) 
                if dp[j] + cost < dp[i]:
                    dp[i] = dp[j] + cost
                    prev[i] = j

    positions = []
    i = n
    while i > 0:
        j = prev[i]
        if j != -1 and i != j and j != 0:
            positions.append(j)
        i = j
    return sorted(positions)

In [388]:
dinamic_separate_freq("книгавхорошемсостоянии") #правильно разбили

[5, 6, 13]

In [389]:
dinamic_separate_freq("купитьайфон14промакс") #тут тоже верно

[6, 11, 13, 16]

In [387]:
def process_dataset(lines):
    """
    Обрабатывает массив строк, добавляет id и predicted_positions, возвращает DataFrame
    """
    df = pd.DataFrame({'id': range(len(lines)), 'input_string': lines})
    
    df['predicted_positions'] = df['input_string'].apply(
        lambda x: dinamic_separate_freq(x)
    )
    
    df = df.drop(columns=['input_string'])
    
    return df


result_df = process_dataset(lines)

output_file = "output2.csv"
result_df.to_csv(output_file, index=False, encoding='utf-8')

Замечание: метод is_real_word достаточно медленный, он замедляет работу метода с 5сек до минуты, но улучшает разбиение (без него алгоритм предпочитает не разбивать длинные строки, изначально is_real_word не использовался, алгоритм выполнился за 5 секунд, но в выводе я заметил очевидные ошибки)

Значение F1 - 90.76% против 81.19% в алгоритме без частот